<a href="https://colab.research.google.com/github/simgyojin/2020-Bigdata-Finance-Challenge/blob/main/%EC%99%B8%EB%B6%80%EB%8D%B0%EC%9D%B4%ED%84%B0/%EC%BD%94%EB%93%9C/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC_%EB%A9%B4%EC%A0%81_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#위키백과에서 면적 크롤링
###행정동 별 면적을 얻기 위해 위키백과에 나와있는 면적을 토대로 행정동 멸 인구 밀도 계산

In [27]:
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from google.colab import files
import re
get_num = re.compile('\d+')
#files.upload()

In [77]:
#전국 행정동 이름이 나와있는 csv파일
district_df = pd.read_csv('행정동_20200814.csv', encoding='utf-8')
district_list = list(district_df['읍면동명'])

#인구밀도 가공 csv파일
population_df = pd.read_csv('인구밀도_가공_2019기준.csv', encoding='utf-8')

In [121]:
#중복되는 행정동에는 시군구명 붙이기
real_district_list=[]
for n, district in enumerate(district_list):
  if str(district) != 'nan':    #nan인 경우 제외
    if district_list.count(district) >=2:   #행정동이 동명인 경우
      if len(str(district_df['시군구명'][n]).split()) >= 2:  #수원시 팔달구 같이 시군구명이 되어 있으면 수원시만 가져옴
        real_district_list.append('{} ({})'.format(district,district_df['시군구명'][n].split()[0]))
      else:
        if str(district_df['시도명'][n])[-1] == '시':       #특별시, 자치시와 같은 경우 위키백과에서 제대로된 데이터 가져오기 위해 사용
          real_district_list.append('{} ({})'.format(district,str(district_df['시도명'][n]).split()[0][:2]))
        else:
          real_district_list.append('{} ({})'.format(district,district_df['시군구명'][n]))
    else:
      real_district_list.append(district)

In [68]:
#위키 백과에서 행정동 면적을 크롤링 하는 코드
#인구밀도가 나와있는 지역도 있지만, 없는 지역도 존재하여 면적만 크롤링
def get_area(district_name):
  url = 'https://ko.wikipedia.org/wiki/{}'.format(district_name)
  driver.get(url)
  area = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table/tbody/tr[4]/td').text
  return area

In [69]:
#get_area test
url = 'https://ko.wikipedia.org/wiki/{}'.format('임동')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver.get(url)
area = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table/tbody/tr[4]/td').text
print(area)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


1.22 km2


In [ ]:
#chome driver사용
'''
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
'''
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

#지역과 지역의 면적을 담을 dictionary
district_area_dic={}

#district_list내의 지역의 면적을 크롤링 real_district_list의 length:3558
for n, district in enumerate(real_district_list):
  try:
    district_area_dic[district]=get_area(district)
    print('{}번째: {} is doing'.format(n, district))
  except:
    try:
      area = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div[1]/ul/li[1]/b/a')
      last_url = area.get_attribute("href").split('/')[-1]
      district_area_dic[district]=get_area(last_url)
      print('{}번째: {} is doing'.format(n, district))
    except:
      print('{}번째: {} is None'.format(n, district))

In [111]:
district_df.head()

,행정동코드,시도명,시군구명,읍면동명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,19880423,NaN
2,1111051500,서울특별시,종로구,청운효자동,20081101,NaN
3,1111053000,서울특별시,종로구,사직동,19880423,NaN
4,1111054000,서울특별시,종로구,삼청동,19880423,NaN


In [109]:
print(len(population_df))
print(len(real_district_list))

3273
3558
